In [5]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [7]:
# 1. Carregar Dados Enriquecidos (V3)
df = pd.read_csv(f"{base_path}df_modelagem_v3.csv", index_col="data", parse_dates=True)

targets = {
    "PF": "inad_pf_tot",
    "PJ": "inad_pj_tot",
    "Rural_PF": "inad_rd_pf_cr_rur_tot", 
    "Rural_PJ": "inad_rd_pj_cr_rur_tot"  
}

# Colunas a remover da base de features (X)
all_targets = list(targets.values())
cols_to_drop_base = [t for t in all_targets if t in df.columns] + ["presidente"]

for nome_modelo, target_col in targets.items():
    if target_col not in df.columns:
        continue
    
    print(f"Treinando Modelo Delta: {nome_modelo}...")
    
    df_temp = df.copy()
    
    # --- A MUDANÇA MÁGICA ---
    # 1. Criar o Target Delta (Diferença do mês atual - mês anterior)
    df_temp["target_diff"] = df_temp[target_col] - df_temp[target_col].shift(1)
    
    # 2. Criar Features de Lag (Inércia ainda importa para explicar volatilidade)
    df_temp["target_lag_1"] = df_temp[target_col].shift(1)
    
    # Removemos nulos gerados pelos lags e diffs
    df_temp = df_temp.dropna()
    
    # SEPARAR X e y
    # O Y agora é a DIFERENÇA, não o valor absoluto
    y = df_temp["target_diff"]
    
    # O X continua sendo as variáveis econômicas + o valor anterior (como ponto de partida)
    X = df_temp.drop(columns=cols_to_drop_base + [target_col, "target_diff"])
    
    # Padronização
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Treinar
    # Usamos menos árvores e limitamos a profundidade para evitar overfitting em ruído
    model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
    model.fit(X_scaled, y)
    
    # Salvar
    joblib.dump(model, f"../models/model_{nome_modelo}.pkl")
    joblib.dump(scaler, f"../models/scaler_{nome_modelo}.pkl")
    pd.DataFrame(columns=X.columns).to_csv(f"../models/columns_{nome_modelo}.csv", index=False)
    # Salvamos o último X para o simulador
    X.iloc[-1].to_csv(f"../models/last_values_{nome_modelo}.csv")

print("\nModelos treinados para prever VARIAÇÃO (Delta)!")

Treinando Modelo Delta: PF...
Treinando Modelo Delta: PJ...
Treinando Modelo Delta: Rural_PF...
Treinando Modelo Delta: Rural_PJ...

Modelos treinados para prever VARIAÇÃO (Delta)!
